# Region of Interest Analysis for TEMPEST

This notebook demonstrates how to select and analyze facets based on latitude/longitude coordinates.

**Example:** Bennu OSIRIS-REx candidate landing sites

## Key Features
- Select single facet closest to a lat/lon point
- Select all facets within a radius of a lat/lon point  
- Visualize selections using TEMPEST's existing animation viewer
- Calculate T^4 mean temperatures for selected regions
- Plot diurnal temperature curves vs local time


In [1]:
# Imports
import sys
import os
sys.path.insert(0, '..')

import numpy as np

from src.utilities.config import Config
from src.model.simulation import Simulation
from tempest import read_shape_model, save_shape_model

# Import spatial selection functions
# NOTE: If you get ImportError, restart the kernel and run again
from src.utilities.spatial_selection import (
    latlon_to_cartesian,
    cartesian_to_latlon,
    find_closest_facet,
    find_facets_in_radius,
    calculate_shape_model_center,
    compare_center_methods,
    read_shape_model_geometry_only
)
from src.utilities.plotting.animate_model import animate_model

print("✓ Imports successful")


✓ Imports successful


**Important:** If you get an ImportError above, you need to restart the kernel:
- In Jupyter: Kernel → Restart Kernel
- Then run all cells again from the beginning


## Setup: Change to TEMPEST root directory


In [2]:
# Change to TEMPEST root directory so file paths work correctly
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f"Working directory: {os.getcwd()}")


Working directory: /Users/duncan/Desktop/DPhil/TEMPEST


## 1. Load Shape Model


In [3]:
# Load configuration
config_file = "private/data/config/bennu/bennu_config.yaml"
config = Config(config_file)

# OPTION 1: Use lightweight geometry-only loader (FAST - recommended for large models)
# This only loads geometry (normal, vertices, area, position) without thermal simulation arrays
# Much faster for models with millions of facets
USE_LIGHTWEIGHT_LOADER = True  # Set to False to use full read_shape_model with thermal properties

if USE_LIGHTWEIGHT_LOADER:
    print("Loading shape model (geometry only - lightweight)...")
    shape_model = read_shape_model_geometry_only(config.path_to_shape_model_file)
    print(f"✓ Shape model loaded: {len(shape_model)} facets (geometry only)")
else:
    # OPTION 2: Use full loader with thermal properties (SLOW for large models)
    simulation = Simulation(config)
    print("Loading shape model (full - with thermal properties)...")
    shape_model = read_shape_model(
        config.path_to_shape_model_file,
        simulation.timesteps_per_day,
        config.n_layers,
        config.max_days,
        config.calculate_energy_terms
    )
    print(f"✓ Shape model loaded: {len(shape_model)} facets (full)")

# Store original filename for export
original_shape_filename = os.path.basename(config.path_to_shape_model_file)
original_shape_basename = os.path.splitext(original_shape_filename)[0]  # Remove .stl extension

# Calculate mean radius (shape model coordinates are in km)
radii = [np.linalg.norm(f.position) for f in shape_model]
mean_radius = np.mean(radii)
print(f"Mean radius: {mean_radius:.3f} km ({mean_radius*1000:.2f} m)")


Loading shape model (geometry only - lightweight)...
Reading STL file: private/data/shape_models/Bennu_0.8M_swatch.stl
Found 17,030 facets in file
  Loaded 851 / 17,030 facets (5.0%) - Rate: 93,691 facets/sec - Est. remaining: 0.0 min
  Loaded 1,702 / 17,030 facets (10.0%) - Rate: 98,838 facets/sec - Est. remaining: 0.0 min
  Loaded 2,553 / 17,030 facets (15.0%) - Rate: 102,877 facets/sec - Est. remaining: 0.0 min
  Loaded 3,404 / 17,030 facets (20.0%) - Rate: 96,972 facets/sec - Est. remaining: 0.0 min
  Loaded 4,255 / 17,030 facets (25.0%) - Rate: 99,970 facets/sec - Est. remaining: 0.0 min
  Loaded 5,106 / 17,030 facets (30.0%) - Rate: 101,664 facets/sec - Est. remaining: 0.0 min
  Loaded 5,957 / 17,030 facets (35.0%) - Rate: 103,177 facets/sec - Est. remaining: 0.0 min
  Loaded 6,808 / 17,030 facets (40.0%) - Rate: 104,366 facets/sec - Est. remaining: 0.0 min
  Loaded 7,659 / 17,030 facets (45.0%) - Rate: 105,236 facets/sec - Est. remaining: 0.0 min
  Loaded 8,510 / 17,030 facets (

## 2. Define Region/s of Interest

Modify these coordinates for your own body/sites.


In [9]:
LANDING_SITES = {
    'Candidate': {
        'lat': 12.5,
        'lon': 130.0,
        'description': 'Best site found for data'
    }
}

# Shape model coordinates are in kilometers
BENNU_RADIUS = 0.245  # kilometers - mean radius (245 m)
SITE_RADIUS = 0.025   # kilometers - search radius for circular region (75 m)

print("Landing sites defined:")
for name, info in LANDING_SITES.items():
    print(f"  {name}: {info['lat']}°N, {info['lon']}°E")


Landing sites defined:
  Candidate: 12.5°N, 130.0°E


## 3. Select Facets for Each Region of Interst

Two selection methods:
1. **`find_closest_facet()`** - Always returns THE single closest facet
2. **`find_facets_in_radius()`** - Returns ALL facets within a circular region (can return 0 if none nearby)


In [10]:
site_selections = {}

print("Region of Interest Facet Selection")
print("=" * 80)

for site_name, site_info in LANDING_SITES.items():
    lat = site_info['lat']
    lon = site_info['lon']
    
    # Method 1: Find closest facet (always returns 1)
    closest_id, closest_dist = find_closest_facet(shape_model, lat, lon, BENNU_RADIUS)
    closest_pos = shape_model[closest_id].position
    closest_lat, closest_lon = cartesian_to_latlon(closest_pos)
    
    # Method 2: Find all facets within radius (can return 0)
    # Debug: Check parameters before calling
    print(f"  DEBUG: Calling find_facets_in_radius with:")
    print(f"    lat={lat}, lon={lon}, body_radius={BENNU_RADIUS}, search_radius={SITE_RADIUS}")
    facets_in_radius = find_facets_in_radius(shape_model, lat, lon, BENNU_RADIUS, SITE_RADIUS)
    print(f"  DEBUG: Returned {len(facets_in_radius)} facets")
    
    # Additional debug: Check a few distances to see if they make sense
    if len(facets_in_radius) == len(shape_model):
        print(f"  ⚠️  ERROR: All facets selected! Checking why...")
        # Get the reference position (closest facet position)
        closest_pos = shape_model[closest_id].position
        print(f"  DEBUG: Reference position (closest facet): {closest_pos}")
        print(f"  DEBUG: Search radius: {SITE_RADIUS} km ({SITE_RADIUS*1000:.1f} m)")
        # Check a few random facets to see their distances
        import random
        sample_ids = random.sample(range(len(shape_model)), min(10, len(shape_model)))
        max_dist = 0
        min_dist = float('inf')
        for fid in sample_ids:
            dist = np.linalg.norm(shape_model[fid].position - closest_pos)
            max_dist = max(max_dist, dist)
            min_dist = min(min_dist, dist)
            print(f"    Facet {fid}: distance = {dist:.4f} km ({dist*1000:.2f} m)")
        print(f"    Min distance in sample: {min_dist:.4f} km ({min_dist*1000:.2f} m)")
        print(f"    Max distance in sample: {max_dist:.4f} km ({max_dist*1000:.2f} m) (search radius: {SITE_RADIUS} km = {SITE_RADIUS*1000:.1f} m)")
        if max_dist > SITE_RADIUS * 10:
            print(f"    ⚠️  Distances are much larger than search radius - something is wrong!")
    elif len(facets_in_radius) > 0:
        # Check distances of first few and last few
        closest_pos = shape_model[closest_id].position
        sample_ids = facets_in_radius[:5] + facets_in_radius[-5:]
        print(f"  DEBUG: Sample distances from reference point:")
        for fid in sample_ids:
            dist = np.linalg.norm(shape_model[fid].position - closest_pos)
            print(f"    Facet {fid}: {dist:.4f} km ({dist*1000:.2f} m)")
    
    # Store results
    site_selections[site_name] = {
        'target_lat': lat,
        'target_lon': lon,
        'closest_facet_id': closest_id,
        'closest_distance': closest_dist,
        'facets_in_radius': facets_in_radius,
        'description': site_info['description']
    }
    
    # Print summary
    print(f"\n{site_name}:")
    print(f"  Description: {site_info['description']}")
    print(f"  Target: {lat}°N, {lon}°E")
    print(f"  Closest facet: {closest_id}")
    print(f"  Closest position: {closest_lat:.1f}°N, {closest_lon:.1f}°E")
    print(f"  Distance to closest: {closest_dist:.4f} km ({closest_dist*1000:.2f} m)")
    print(f"  Facets within {SITE_RADIUS} km ({SITE_RADIUS*1000:.1f} m) radius: {len(facets_in_radius)}")
    
    if facets_in_radius:
        total_area = sum(shape_model[fid].area for fid in facets_in_radius)
        print(f"  Total area of region: {total_area:.2f} m²")
        # Debug: Check if selection seems reasonable
        if len(facets_in_radius) == len(shape_model):
            print(f"  ⚠️  WARNING: Selected ALL facets! This seems wrong for a {SITE_RADIUS} km ({SITE_RADIUS*1000:.1f} m) radius.")
            print(f"     Expected ~{np.pi * (SITE_RADIUS*1000)**2:.0f} m² area, got {total_area:.0f} m²")
            print(f"     Check that SITE_RADIUS ({SITE_RADIUS} km = {SITE_RADIUS*1000:.1f} m) is correct and much smaller than body radius ({BENNU_RADIUS} km = {BENNU_RADIUS*1000:.1f} m)")
        elif len(facets_in_radius) > len(shape_model) * 0.1:  # More than 10% of facets
            print(f"  ⚠️  WARNING: Selected {100*len(facets_in_radius)/len(shape_model):.1f}% of all facets.")
            print(f"     This seems high for a {SITE_RADIUS} km ({SITE_RADIUS*1000:.1f} m) radius on a {BENNU_RADIUS} km ({BENNU_RADIUS*1000:.1f} m) radius body.")
    elif closest_dist > SITE_RADIUS:
        print(f"  No facets within radius (increase SITE_RADIUS to >{closest_dist:.4f} km ({closest_dist*1000:.1f} m) to include closest)")

print("\n" + "=" * 80)


Region of Interest Facet Selection
  DEBUG: Calling find_facets_in_radius with:
    lat=12.5, lon=130.0, body_radius=0.245, search_radius=0.025
  DEBUG: Returned 1790 facets
  DEBUG: Sample distances from reference point:
    Facet 8761: 0.0000 km (0.00 m)
    Facet 7638: 0.0009 km (0.86 m)
    Facet 11456: 0.0012 km (1.18 m)
    Facet 12181: 0.0015 km (1.46 m)
    Facet 7637: 0.0015 km (1.47 m)
    Facet 3576: 0.0250 km (24.96 m)
    Facet 3974: 0.0250 km (24.97 m)
    Facet 13072: 0.0250 km (24.98 m)
    Facet 10147: 0.0250 km (24.99 m)
    Facet 12798: 0.0250 km (24.99 m)

Candidate:
  Description: Best site found for data
  Target: 12.5°N, 130.0°E
  Closest facet: 8761
  Closest position: 13.4°N, 129.7°E
  Distance to closest: 0.0064 km (6.43 m)
  Facets within 0.025 km (25.0 m) radius: 1790
  Total area of region: 0.00 m²
  ⚠️  WARNING: Selected 10.5% of all facets.
     This seems high for a 0.025 km (25.0 m) radius on a 0.245 km (245.0 m) radius body.



In [11]:
print("ROI Facet IDs - Detailed List")
print("=" * 80)

for site_name, data in site_selections.items():
    print(f"\n{site_name}:")
    print(f"  Target: {data['target_lat']}°N, {data['target_lon']}°E")
    print(f"  Description: {data['description']}")
    print(f"  Closest facet: {data['closest_facet_id']}")
    
    facets = data['facets_in_radius']
    print(f"  Number of facets in radius: {len(facets)}")
    
    if facets:
        total_area = sum(shape_model[fid].area for fid in facets)
        print(f"  Total area: {total_area:.2f} m²")
        print(f"  Expected area (π×25²): {np.pi * 25**2:.2f} m²")
        
        # Print facet IDs in a nice format
        print(f"\n  All facet IDs ({len(facets)} total):")
        # Print 10 IDs per line
        for i in range(0, len(facets), 10):
            batch = facets[i:i+10]
            id_str = ", ".join(f"{fid:5d}" for fid in batch)
            print(f"    {id_str}")
    else:
        print("  No facets found within radius!")
    
print("\n" + "=" * 80)


ROI Facet IDs - Detailed List

Candidate:
  Target: 12.5°N, 130.0°E
  Description: Best site found for data
  Closest facet: 8761
  Number of facets in radius: 1790
  Total area: 0.00 m²
  Expected area (π×25²): 1963.50 m²

  All facet IDs (1790 total):
     8761,  7638, 11456, 12181,  7637,  7639,  3213,  8762,  4706,  5131
    14464, 12180,  9401, 14463,  8404,  1249,  5130, 14462,   883,  4707
    16158, 13478, 10078,  8763, 10079,  5129, 11815,  4693,  9404,  2051
    14474,  1253,  3216,  1250,  3215, 12175, 14821,  2052, 11457, 11814
    10077, 13475,  8023, 10760,  2049, 12176, 11813,  9719,  4694,  6925
     1252, 12536, 14822, 11811, 13474, 11810,  7650,  9718,  3224,  1251
    10752,  7654, 11812, 16165,  3214,  9413,  8416,   143,  9403,  8029
    12537,  8403, 12174,  4370,  6929, 12183, 16166, 15180,  7655, 11095
    12535, 11094, 13150, 11808, 15179, 13473,  1240,  8025,  9111, 11843
    10781,  9402,  8411, 10782, 11809,  2050, 12182, 11841, 13170,  7653
    13497,  6926

In [12]:
# Choose which facets to export:
# Option 1: Export a specific site (set site_name to one of the keys in site_selections)
# Option 2: Export all selected facets from all sites combined
# Option 3: Export a custom list of facet IDs

# Configuration
EXPORT_SITE = 'Candidate'  # Set to site name (e.g., 'Nightingale') to export that site only, or None for all sites
CUSTOM_FACET_IDS = None  # Set to a list of facet IDs to export custom selection, or None to use site selections

# Output filename (includes original shape file name)
if EXPORT_SITE:
    output_basename = f"{original_shape_basename}_roi_{EXPORT_SITE.lower()}"
    selected_facet_ids = set(site_selections[EXPORT_SITE]['facets_in_radius'])
    print(f"Exporting shape model for site: {EXPORT_SITE}")
elif CUSTOM_FACET_IDS:
    output_basename = f"{original_shape_basename}_roi_custom"
    selected_facet_ids = set(CUSTOM_FACET_IDS)
    print(f"Exporting shape model with {len(selected_facet_ids)} custom facets")
else:
    # Combine all facets from all sites
    output_basename = f"{original_shape_basename}_roi_all_sites"
    selected_facet_ids = set()
    for site_name, data in site_selections.items():
        selected_facet_ids.update(data['facets_in_radius'])
    print(f"Exporting shape model with facets from all sites")
    print(f"  Sites included: {', '.join(site_selections.keys())}")

output_filename = f"output/{output_basename}.stl"
csv_filename = f"output/{output_basename}_facet_ids.csv"

print(f"  Total unique facets to export: {len(selected_facet_ids)}")

# Create filtered shape model with only selected facets
filtered_shape_model = [shape_model[fid] for fid in sorted(selected_facet_ids)]

print(f"  Filtered shape model created")

# Create output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

# Export the filtered shape model as STL
# Note: save_shape_model needs config, but if using lightweight loader we might not have it
# Create a minimal config-like object if needed
try:
    save_shape_model(filtered_shape_model, output_filename, config)
except (NameError, AttributeError):
    # If config doesn't exist (lightweight loader), save manually
    with open(output_filename, 'w') as f:
        f.write("solid model\n")
        for facet in filtered_shape_model:
            f.write(f"facet normal {' '.join(map(str, facet.normal))}\n")
            f.write("  outer loop\n")
            for vertex in facet.vertices:
                f.write(f"    vertex {' '.join(map(str, vertex))}\n")
            f.write("  endloop\n")
            f.write("endfacet\n")
        f.write("endsolid model\n")
    print(f"  (Saved using lightweight method)")

# Export facet IDs as CSV
sorted_facet_ids = sorted(selected_facet_ids)
np.savetxt(csv_filename, sorted_facet_ids, fmt='%d', delimiter=',', 
           header='facet_id', comments='')

print(f"\n✓ Shape model exported to: {output_filename}")
print(f"✓ Facet IDs exported to: {csv_filename}")
print(f"  Original facets: {len(shape_model)}")
print(f"  Exported facets: {len(filtered_shape_model)}")
print(f"  Reduction: {100 * (1 - len(filtered_shape_model) / len(shape_model)):.1f}%")


Exporting shape model for site: Candidate
  Total unique facets to export: 1790
  Filtered shape model created
Shape model saved to output/Bennu_0.8M_swatch_roi_candidate.stl

✓ Shape model exported to: output/Bennu_0.8M_swatch_roi_candidate.stl
✓ Facet IDs exported to: output/Bennu_0.8M_swatch_roi_candidate_facet_ids.csv
  Original facets: 17030
  Exported facets: 1790
  Reduction: 89.5%


In [13]:
# Helper function: Select facets within radius of a Cartesian position
def find_facets_in_radius_from_position(shape_model, center_position, search_radius_km):
    """
    Find all facets within a radius of a given Cartesian position.
    This is useful when you want to select facets using a Cartesian position
    instead of lat/lon coordinates.
    
    Args:
        shape_model: List of Facet objects (coordinates in km)
        center_position: Cartesian position [x, y, z] to search around (in km)
        search_radius_km: Radius of search circle in kilometers
        
    Returns:
        List of facet IDs within the search radius, sorted by distance
    """
    facets_in_radius = []
    
    for i, facet in enumerate(shape_model):
        distance = np.linalg.norm(facet.position - center_position)
        if distance <= search_radius_km:
            facets_in_radius.append((i, distance))
    
    # Sort by distance (closest first)
    facets_in_radius.sort(key=lambda x: x[1])
    
    # Return just the facet IDs
    return [fid for fid, _ in facets_in_radius]

# Example usage (commented out):
# If you want to select facets using a Cartesian position:
# center_pos = np.array([x, y, z])  # Your Cartesian position (in km)
# facets = find_facets_in_radius_from_position(shape_model, center_pos, 0.025)  # 25 m = 0.025 km
